# 📊 Fairness Algorithm Results Processing Pipeline

This notebook provides a complete pipeline for processing experimental results from fairness algorithm comparisons.

## 🎯 **Objectives:**
- Load and standardize experimental results from multiple CSV files
- Integrate runtime data for complete analysis
- Generate publication-ready tables with LaTeX support
- Compare FoldOpt, PTO, and Two-Stage methods across different fairness metrics

## 📋 **Output:**
- **Regular HTML Tables** for notebook viewing
- **LaTeX Tables** for academic publications
- **Statistical Analysis** with mean ± std formatting

---

## 🔧 Configuration and Imports

In [69]:
import pandas as pd
import numpy as np
import os
from typing import Dict, List

# Configuration
RESULTS_DIR = r'E:\myREPO\Fairness-Decision-Focused-Loss\Organized-FDFL\notebooks\results'
RUNTIME_FILE = 'times-foldopt.csv'

print("📦 Imports and configuration completed successfully!")

📦 Imports and configuration completed successfully!


## 🏗️ Data Processing Classes

In [70]:
class ResultsProcessor:
    """Comprehensive results processing pipeline for fairness algorithm experiments."""
    
    def __init__(self, results_dir: str):
        self.results_dir = results_dir
        self.dataframes = {}
        self.master_df = None
        
        # Column standardization mapping
        self.base_rename_map = {
            'loss': 'Training Loss mean', 'mse': 'Prediction MSE mean',
            'loss_std': 'Training Loss std', 'mse_std': 'Prediction MSE std',
            'regret': 'Decision Regret mean', 'regret_std': 'Decision Regret std',
            'fairness_std': 'Prediction Fairness std',
            'Fairness Penalty std': 'Prediction Fairness std',
            'training_time': 'Training Time mean', 'training_time_std': 'Training Time std',
            'Predictor mean': 'Predictor',
            
            # G0/G1 Objective columns
            'G0 Decision Obj mean': 'G0 Objective mean', 'G0 Objective': 'G0 Objective mean',
            'G0_decision_obj': 'G0 Objective mean', 'G0 Decision Obj std': 'G0 Objective std',
            'G0_objective_std': 'G0 Objective std', 'G0_decision_obj_std': 'G0 Objective std',
            'G1 Decision Obj mean': 'G1 Objective mean', 'G1 Objective': 'G1 Objective mean',
            'G1_decision_obj': 'G1 Objective mean', 'G1 Decision Obj std': 'G1 Objective std',
            'G1_objective_std': 'G1 Objective std', 'G1_decision_obj_std': 'G1 Objective std',
            
            # G0/G1 MSE columns
            'G0 MSE mean': 'G0 MSE mean', 'G0 MSE': 'G0 MSE mean', 'G0_mse': 'G0 MSE mean',
            'G0 MSE std': 'G0 MSE std', 'G0_mse_std': 'G0 MSE std',
            'G1 MSE mean': 'G1 MSE mean', 'G1 MSE': 'G1 MSE mean', 'G1_mse': 'G1 MSE mean',
            'G1 MSE std': 'G1 MSE std', 'G1_mse_std': 'G1 MSE std',
            
            # G0/G1 Fairness columns
            'G0 True Benefit mean': 'G0 True Benefit mean', 'G0 Fairness': 'G0 True Benefit mean',
            'G0 True Benefit std': 'G0 True Benefit std', 'G0_fairness_std': 'G0 True Benefit std',
            'G1 True Benefit mean': 'G1 True Benefit mean', 'G1 Fairness': 'G1 True Benefit mean',
            'G1 True Benefit std': 'G1 True Benefit std', 'G1_fairness_std': 'G1 True Benefit std'
        }
    
    def load_all_csvs(self) -> Dict[str, pd.DataFrame]:
        """Load all CSV files from the results directory."""
        print("📂 Loading CSV files...")
        
        csv_files = [f for f in os.listdir(self.results_dir) if f.endswith('.csv')]
        
        for csv_file in csv_files:
            file_path = os.path.join(self.results_dir, csv_file)
            df = pd.read_csv(file_path)
            self.dataframes[csv_file] = df
            print(f"   ✓ {csv_file}: {df.shape}")
        
        print(f"📊 Loaded {len(csv_files)} CSV files")
        return self.dataframes
    
    def standardize_dataframe(self, df: pd.DataFrame, filename: str) -> pd.DataFrame:
        """Standardize column names and add method/predictor identifiers."""
        temp_df = df.copy()
        
        # Apply base rename mapping
        temp_df.rename(columns=self.base_rename_map, inplace=True)
        
        # File-specific column handling
        if 'pto' in filename:
            temp_df.rename(columns={'fairness': 'Prediction Fairness mean'}, inplace=True)
            temp_df['Method'] = 'FPTO'
        elif 'foldopt' in filename:
            temp_df.rename(columns={
                'Fairness Penalty mean': 'Prediction Fairness mean',
                'FairnessType': 'Fairness'
            }, inplace=True)
            temp_df['Method'] = 'FoldOpt'
        else:  # res-lr.csv and res-nn.csv
            if 'Grad Method' in temp_df.columns:
                temp_df['Method'] = temp_df['Grad Method']
            else:
                temp_df['Method'] = 'FDFL'
        
        # Add predictor information
        if 'nn' in filename:
            temp_df['Predictor'] = 'NN'
        elif 'lr' in filename:
            temp_df['Predictor'] = 'LR'
        
        # Clean up
        if 'Unnamed: 0' in temp_df.columns:
            temp_df.drop(columns=['Unnamed: 0'], inplace=True)
        
        return temp_df
    
    def create_master_dataframe(self) -> pd.DataFrame:
        """Create unified master dataframe from all loaded files."""
        print("🔄 Creating master dataframe...")
        
        processed_dfs = []
        for filename, df in self.dataframes.items():
            if 'times-' not in filename:  # Skip runtime files for now
                standardized_df = self.standardize_dataframe(df, filename)
                processed_dfs.append(standardized_df)
                print(f"   ✓ Processed {filename}")
        
        self.master_df = pd.concat(processed_dfs, ignore_index=True, sort=False)
        print(f"📊 Master dataframe created: {self.master_df.shape}")
        return self.master_df
    
    def integrate_runtime_data(self, runtime_filename: str = 'times-foldopt.csv'):
        """Integrate FoldOpt runtime data into master dataframe."""
        if runtime_filename not in self.dataframes:
            print(f"⚠️  Runtime file {runtime_filename} not found")
            return
        
        print("⏱️  Integrating runtime data...")
        runtime_df = self.dataframes[runtime_filename].copy()
        
        # Standardize runtime data
        if 'FairnessType' in runtime_df.columns:
            runtime_df.rename(columns={'FairnessType': 'Fairness'}, inplace=True)
        if 'Mean Runtime' in runtime_df.columns:
            runtime_df.rename(columns={
                'Mean Runtime': 'Training Time mean',
                'Std Runtime': 'Training Time std'
            }, inplace=True)
        
        # Fix predictor naming in runtime data
        runtime_df['Predictor'] = runtime_df['Predictor'].replace({
            'LinearRegressionModel': 'LR', 
            'FairRiskPredictor': 'NN'
        })
        
        runtime_df['Group'] = runtime_df['Group'].astype(bool)
        
        # Update FoldOpt rows with runtime data
        merge_cols = ['Group', 'Fairness', 'Alpha', 'Lambda', 'Predictor']
        runtime_for_merge = runtime_df[merge_cols + ['Training Time mean', 'Training Time std']].copy()
        
        foldopt_mask = self.master_df['Method'] == 'FoldOpt'
        updated_count = 0
        
        for idx in self.master_df[foldopt_mask].index:
            row = self.master_df.loc[idx]
            
            # Find matching runtime data
            match_mask = (
                (runtime_for_merge['Group'] == row['Group']) &
                (runtime_for_merge['Fairness'] == row['Fairness']) &
                (runtime_for_merge['Alpha'] == row['Alpha']) &
                (runtime_for_merge['Lambda'] == row['Lambda']) &
                (runtime_for_merge['Predictor'] == row['Predictor'])
            )
            
            matching_runtime = runtime_for_merge[match_mask]
            
            if len(matching_runtime) > 0:
                time_data = matching_runtime.iloc[0]
                self.master_df.loc[idx, 'Training Time mean'] = time_data['Training Time mean']
                self.master_df.loc[idx, 'Training Time std'] = time_data['Training Time std']
                updated_count += 1
        
        print(f"   ✓ Updated {updated_count} FoldOpt rows with runtime data")
        print(f"📊 Final master dataframe: {self.master_df.shape}")
        
    def get_summary(self) -> Dict[str, pd.Series]:
        """Get summary statistics of the master dataframe."""
        summary = {
            'Has_Results': self.master_df.groupby('Method').agg({
                'Decision Regret mean': lambda x: (~pd.isna(x)).sum()
            })['Decision Regret mean'],
            'Has_Runtime': self.master_df.groupby('Method').agg({
                'Training Time mean': lambda x: (~pd.isna(x)).sum()
            })['Training Time mean']
        }
        return summary

print("🏗️  ResultsProcessor class defined successfully!")

🏗️  ResultsProcessor class defined successfully!


In [71]:
class TableGenerator:
    """Generate publication-ready tables from processed results."""
    
    def __init__(self, master_df: pd.DataFrame):
        self.master_df = master_df
    
    def format_with_std(self, df: pd.DataFrame, value_col: str, index: List[str], 
                       columns: str, aggfunc='mean') -> pd.DataFrame:
        """Create pivot table with 'mean ± std' format with dollar signs for math mode."""
        mean_col = value_col
        std_col = value_col.replace('mean', 'std')
        
        if std_col not in df.columns:
            mean_table = pd.pivot_table(df, values=mean_col, index=index, 
                                      columns=columns, aggfunc=aggfunc)
            return mean_table.map(lambda x: f'${x:.4f}$' if pd.notna(x) else '$-$')
        
        mean_table = pd.pivot_table(df, values=mean_col, index=index, 
                                  columns=columns, aggfunc=aggfunc)
        std_table = pd.pivot_table(df, values=std_col, index=index, 
                                 columns=columns, aggfunc=aggfunc)
        
        formatted_table = mean_table.map(lambda x: f'${x:.4f}$' if pd.notna(x) else '$-$') + \
                         " ± " + \
                         std_table.map(lambda x: f'${x:.4f}$' if pd.notna(x) else '$-$')
        
        return formatted_table.replace('$-$ ± $-$', '$-$', regex=False)
    
    def to_latex(self, table: pd.DataFrame, table_name: str, group_setting: bool, fairness_type: str) -> str:
        """Convert pivot table to LaTeX format with centered alignment."""
        
        # Create caption and label
        group_str = "group alpha fairness objective" if group_setting else "individual alpha fairness objective"
        caption = f"{table_name} results ({group_str}, {fairness_type.upper()} fairness)"
        label_group = "group" if group_setting else "individual"
        label = f"tab:{table_name.lower().replace(' ', '_')}_{fairness_type}_{label_group}_alpha"
        
        # Start LaTeX table
        latex_lines = [
            "\\begin{table}[htbp]",
            "\\centering",
            f"\\caption{{{caption}}}",
            f"\\label{{{label}}}",
            "\\footnotesize",  # Make font smaller for better fit
        ]
        
        # Calculate number of columns (index levels + data columns)
        n_index_cols = len(table.index.names) if hasattr(table.index, 'names') else 1
        n_data_cols = len(table.columns)
        
        # Column specification - left align for index columns, center align for data columns
        col_spec = "l" * n_index_cols + "c" * n_data_cols
        latex_lines.append(f"\\begin{{tabular}}{{{col_spec}}}")
        latex_lines.append("\\toprule")
        
        # Header row
        index_names = []
        if hasattr(table.index, 'names'):
            index_names = [name if name else "" for name in table.index.names]
        else:
            index_names = [""]
        
        # Create header with dollar signs for alpha values
        header_parts = index_names + [f"$\\alpha = {col}$" for col in table.columns]
        header_line = " & ".join(header_parts) + " \\\\"
        latex_lines.append(header_line)
        latex_lines.append("\\midrule")
        
        # Data rows
        for idx, row in table.iterrows():
            # Handle multi-index
            if isinstance(idx, tuple):
                row_parts = list(idx)
            else:
                row_parts = [str(idx)]
            
            # Format algorithm names for LaTeX
            for i, part in enumerate(row_parts):
                if part == 'closed-form':
                    row_parts[i] = 'FDFL-CF'
                elif part == 'finite-diff':
                    row_parts[i] = 'FDFL-FD'
                elif part == 'FoldOpt':
                    row_parts[i] = 'FoldOpt'
                elif part == 'FPTO':
                    row_parts[i] = 'FPTO'
                else:
                    row_parts[i] = str(part)
            
            # Add data values (already formatted with dollar signs from format_with_std)
            data_parts = [str(val) if pd.notna(val) else "$-$" for val in row.values]
            
            # Combine all parts
            all_parts = row_parts + data_parts
            row_line = " & ".join(all_parts) + " \\\\"
            latex_lines.append(row_line)
        
        # End table
        latex_lines.extend([
            "\\bottomrule",
            "\\end{tabular}",
            "\\end{table}",
            ""
        ])
        
        return "\n".join(latex_lines)
    
    def generate_tables(self, group_setting: bool, fairness_type: str, 
                       alpha_values: List[float] = [0.5, 2], output_latex: bool = False):
        """Generate complete set of results tables."""
        
        # Determine lambda values
        lambda_values = {
            'mad': [0, 0.05, 0.5],
            'atkinson': [0, 1, 5]
        }.get(fairness_type, [0])
        
        group_desc = "GROUP ALPHA FAIRNESS" if group_setting else "INDIVIDUAL ALPHA FAIRNESS"
        print(f"{'='*80}")
        print(f"    📋 RESULTS FOR: {group_desc}, FAIRNESS = {fairness_type.upper()}")
        print(f"{'='*80}")
        
        # Filter data
        df = self.master_df[
            (self.master_df['Group'] == group_setting) &
            (self.master_df['Fairness'].str.contains(fairness_type, case=False, na=False)) &
            (self.master_df['Lambda'].isin(lambda_values)) &
            (self.master_df['Alpha'].isin(alpha_values))
        ].copy()
        
        if df.empty:
            print(f"⚠️  No data found for specified filters")
            return
        
        # Create Algorithm column
        df['Algorithm'] = df['Method']
        two_stage_mask = df['Method'] == 'Two-Stage'
        if two_stage_mask.any():
            df.loc[two_stage_mask, 'Algorithm'] = df.loc[two_stage_mask, 'Grad Method']
        
        table_index = ['Algorithm', 'Predictor', 'Fairness', 'Lambda']
        
        # Generate tables
        tables = {
            'Decision Regret': 'Decision Regret mean',
            'Prediction MSE': 'Prediction MSE mean', 
            'Prediction Fairness': 'Prediction Fairness mean',
            'Training Time': 'Training Time mean'
        }
        
        for i, (table_name, metric_col) in enumerate(tables.items(), 1):
            print(f"\n------ {i}. {table_name} (mean ± std) ------")
            
            # Special handling for FoldOpt MSE calculation
            if metric_col == 'Prediction MSE mean':
                n0, n1 = 4422, 578
                foldopt_mask = (df['Method'] == 'FoldOpt') & (df[metric_col].isna())
                if not df.loc[foldopt_mask].empty:
                    g0_mse_mean = df.loc[foldopt_mask, 'G0 MSE mean']
                    g1_mse_mean = df.loc[foldopt_mask, 'G1 MSE mean']
                    df.loc[foldopt_mask, metric_col] = (g0_mse_mean * n0 + g1_mse_mean * n1) / (n0 + n1)
                    
                    g0_mse_std = df.loc[foldopt_mask, 'G0 MSE std']
                    g1_mse_std = df.loc[foldopt_mask, 'G1 MSE std']
                    df.loc[foldopt_mask, 'Prediction MSE std'] = np.sqrt((g0_mse_std**2 * n0 + g1_mse_std**2 * n1) / (n0 + n1))
            
            table = self.format_with_std(df, metric_col, table_index, 'Alpha')
            
            if output_latex:
                latex_code = self.to_latex(table, table_name, group_setting, fairness_type)
                print(f"\nLaTeX code for {table_name}:")
                print(latex_code)
            else:
                display(table)
        
        print(f"\n{'='*80}")
    
    def generate_all_latex_tables(self, alpha_values: List[float] = [0.5, 2]):
        """Generate LaTeX code for all table configurations."""
        
        fairness_configs = [
            ('mad', True), ('mad', False), 
            ('atkinson', True), ('atkinson', False)
        ]
        
        print("🎯 Generating LaTeX code for all publication-ready tables...")
        print("="*100)
        
        all_latex = []
        
        for fairness_type, group_setting in fairness_configs:
            group_desc = "group alpha fairness" if group_setting else "individual alpha fairness"
            print(f"\n📋 Processing: {fairness_type.upper()} fairness, {group_desc} objective")
            latex_tables = self._generate_latex_for_config(group_setting, fairness_type, alpha_values)
            all_latex.extend(latex_tables)
        
        complete_latex = "\n\n".join(all_latex)
        
        print(f"\n✅ Generated LaTeX for {len(all_latex)} tables")
        return complete_latex
    
    def _generate_latex_for_config(self, group_setting: bool, fairness_type: str, 
                                  alpha_values: List[float]) -> List[str]:
        """Generate LaTeX tables for a specific configuration."""
        
        # Determine lambda values
        lambda_values = {
            'mad': [0, 0.05, 0.5],
            'atkinson': [0, 1, 5]
        }.get(fairness_type, [0])
        
        # Filter data
        df = self.master_df[
            (self.master_df['Group'] == group_setting) &
            (self.master_df['Fairness'].str.contains(fairness_type, case=False, na=False)) &
            (self.master_df['Lambda'].isin(lambda_values)) &
            (self.master_df['Alpha'].isin(alpha_values))
        ].copy()
        
        if df.empty:
            return []
        
        # Create Algorithm column
        df['Algorithm'] = df['Method']
        two_stage_mask = df['Method'] == 'Two-Stage'
        if two_stage_mask.any():
            df.loc[two_stage_mask, 'Algorithm'] = df.loc[two_stage_mask, 'Grad Method']
        
        table_index = ['Algorithm', 'Predictor', 'Fairness', 'Lambda']
        
        # Generate tables
        tables = {
            'Decision Regret': 'Decision Regret mean',
            'Prediction MSE': 'Prediction MSE mean', 
            'Prediction Fairness': 'Prediction Fairness mean',
            'Training Time': 'Training Time mean'
        }
        
        latex_tables = []
        
        for table_name, metric_col in tables.items():
            # Special handling for FoldOpt MSE calculation
            if metric_col == 'Prediction MSE mean':
                n0, n1 = 4422, 578
                foldopt_mask = (df['Method'] == 'FoldOpt') & (df[metric_col].isna())
                if not df.loc[foldopt_mask].empty:
                    g0_mse_mean = df.loc[foldopt_mask, 'G0 MSE mean']
                    g1_mse_mean = df.loc[foldopt_mask, 'G1 MSE mean']
                    df.loc[foldopt_mask, metric_col] = (g0_mse_mean * n0 + g1_mse_mean * n1) / (n0 + n1)
                    
                    g0_mse_std = df.loc[foldopt_mask, 'G0 MSE std']
                    g1_mse_std = df.loc[foldopt_mask, 'G1 MSE std']
                    df.loc[foldopt_mask, 'Prediction MSE std'] = np.sqrt((g0_mse_std**2 * n0 + g1_mse_std**2 * n1) / (n0 + n1))
            
            table = self.format_with_std(df, metric_col, table_index, 'Alpha')
            latex_code = self.to_latex(table, table_name, group_setting, fairness_type)
            latex_tables.append(latex_code)
        
        return latex_tables

print("📊 TableGenerator class defined successfully!")

📊 TableGenerator class defined successfully!


## 🚀 Main Execution Pipeline

In [72]:
def create_complete_results_pipeline():
    """Complete pipeline to process all experimental results."""
    
    print("🔄 Starting complete results processing pipeline...")
    print("="*80)
    
    # Initialize processor with results directory
    processor = ResultsProcessor(RESULTS_DIR)
    
    # Load and process all data
    print("📂 Loading all CSV files...")
    csv_files = processor.load_all_csvs()
    print(f"✅ Loaded {len(csv_files)} files: {list(csv_files.keys())}")
    
    # Create master dataframe
    print("\n🔗 Creating master dataframe...")
    master_df = processor.create_master_dataframe()
    print(f"✅ Master dataframe created with shape: {master_df.shape}")
    
    # Integrate runtime data
    print("\n⏱️  Integrating runtime data...")
    processor.integrate_runtime_data(RUNTIME_FILE)
    master_df = processor.master_df  # Update with integrated data
    
    # Show summary
    print("\n📊 Data Summary:")
    summary = processor.get_summary()
    for key, value in summary.items():
        print(f"  • {key}: {value}")
    
    # Initialize table generator
    print("\n📋 Initializing table generator...")
    table_gen = TableGenerator(master_df)
    
    return processor, table_gen, master_df

# Run the complete pipeline
processor, table_gen, master_df = create_complete_results_pipeline()

🔄 Starting complete results processing pipeline...
📂 Loading all CSV files...
📂 Loading CSV files...
   ✓ res-foldopt-lr.csv: (24, 23)
   ✓ res-foldopt-nn.csv: (24, 23)
   ✓ res-lr.csv: (48, 26)
   ✓ res-nn.csv: (48, 26)
   ✓ res-pto-lr.csv: (24, 20)
   ✓ res-pto-nn.csv: (24, 20)
   ✓ times-foldopt.csv: (48, 7)
📊 Loaded 7 CSV files
✅ Loaded 7 files: ['res-foldopt-lr.csv', 'res-foldopt-nn.csv', 'res-lr.csv', 'res-nn.csv', 'res-pto-lr.csv', 'res-pto-nn.csv', 'times-foldopt.csv']

🔗 Creating master dataframe...
🔄 Creating master dataframe...
   ✓ Processed res-foldopt-lr.csv
   ✓ Processed res-foldopt-nn.csv
   ✓ Processed res-lr.csv
   ✓ Processed res-nn.csv
   ✓ Processed res-pto-lr.csv
   ✓ Processed res-pto-nn.csv
📊 Master dataframe created: (192, 29)
✅ Master dataframe created with shape: (192, 29)

⏱️  Integrating runtime data...
⏱️  Integrating runtime data...
   ✓ Updated 48 FoldOpt rows with runtime data
📊 Final master dataframe: (192, 29)

📊 Data Summary:
  • Has_Results: Method

## 📋 Generate Results Tables

Choose your preferred output format:
- **HTML Tables**: For viewing in notebook
- **LaTeX Tables**: For academic publications

In [73]:
# Configuration for table generation
fairness_configs = [
    ('mad', True),     # MAD fairness with group alpha fairness objective
    ('mad', False),    # MAD fairness with individual alpha fairness objective
    ('atkinson', True), # Atkinson fairness with group alpha fairness objective
    ('atkinson', False) # Atkinson fairness with individual alpha fairness objective
]

alpha_values = [0.5, 2]  # Risk aversion parameters

print("🎯 Available table configurations:")
for i, (fairness_type, group_setting) in enumerate(fairness_configs, 1):
    group_str = "group alpha fairness objective" if group_setting else "individual alpha fairness objective"
    print(f"  {i}. {fairness_type.upper()} fairness with {group_str}")
    
print(f"\n📊 Alpha values: {alpha_values}")
print("\n📋 Each configuration generates 4 tables: Decision Regret, Prediction MSE, Prediction Fairness, Training Time")

🎯 Available table configurations:
  1. MAD fairness with group alpha fairness objective
  2. MAD fairness with individual alpha fairness objective
  3. ATKINSON fairness with group alpha fairness objective
  4. ATKINSON fairness with individual alpha fairness objective

📊 Alpha values: [0.5, 2]

📋 Each configuration generates 4 tables: Decision Regret, Prediction MSE, Prediction Fairness, Training Time


### 📊 Generate HTML Tables (for notebook viewing)

In [74]:
# Generate regular HTML tables for notebook viewing
print("🎯 Generating HTML tables for notebook viewing...")
print("="*100)

# Example: Generate tables for MAD fairness with group alpha fairness objective
table_gen.generate_tables(
    group_setting=True,
    fairness_type='mad',
    alpha_values=alpha_values,
    output_latex=False
)

print("\n" + "="*100)
print("💡 To generate other configurations, change the parameters above:")
print("   - group_setting: True (group alpha fairness) / False (individual alpha fairness)")
print("   - fairness_type: 'mad' or 'atkinson'")
print("   - alpha_values: [0.5, 2] or custom list")

🎯 Generating HTML tables for notebook viewing...
    📋 RESULTS FOR: GROUP ALPHA FAIRNESS, FAIRNESS = MAD

------ 1. Decision Regret (mean ± std) ------


Alpha                                                  0.5  \
Algorithm   Predictor Fairness Lambda                        
FPTO        LR        mad      0.00    $0.1019$ ± $0.0002$   
                               0.05    $0.1019$ ± $0.0002$   
                               0.50    $0.1015$ ± $0.0003$   
            NN        mad      0.00    $0.0176$ ± $0.0004$   
                               0.05    $0.0176$ ± $0.0004$   
                               0.50    $0.0179$ ± $0.0011$   
FoldOpt     LR        mad      0.00    $0.1278$ ± $0.0006$   
                               0.05    $0.1278$ ± $0.0006$   
                               0.50    $0.1278$ ± $0.0006$   
            NN        mad      0.00    $0.1325$ ± $0.0039$   
                               0.05    $0.1325$ ± $0.0039$   
                               0.50    $0.1325$ ± $0.0039$   
closed-form LR        mad      0.00    $0.0234$ ± $0.0003$   
                               0.05    $0.0915$ ± $0.0006$   
                               0.50    $0.0916$ ± $0.0005$   
            NN        mad      0.00    $0.0189$ ± $0.0012$   
                               0.05    $0.0735$ ± $0.0015$   
                               0.50    $0.0763$ ± $0.0013$   
finite-diff LR        mad      0.00    $0.0444$ ± $0.0000$   
                               0.05    $0.0727$ ± $0.0002$   
                               0.50    $0.0728$ ± $0.0002$   
            NN        mad      0.00    $0.0223$ ± $0.0035$   
                               0.05    $0.0801$ ± $0.0007$   
                               0.50    $0.0810$ ± $0.0007$   

Alpha                                                  2.0  
Algorithm   Predictor Fairness Lambda                       
FPTO        LR        mad      0.00    $0.4741$ ± $0.0048$  
                               0.05    $0.4740$ ± $0.0046$  
                               0.50    $0.4699$ ± $0.0050$  
            NN        mad      0.00    $0.0983$ ± $0.0018$  
                               0.05    $0.0979$ ± $0.0015$  
                               0.50    $0.0968$ ± $0.0039$  
FoldOpt     LR        mad      0.00    $0.1117$ ± $0.0119$  
                               0.05    $0.1129$ ± $0.0115$  
                               0.50    $0.1129$ ± $0.0115$  
            NN        mad      0.00    $0.1195$ ± $0.0068$  
                               0.05    $0.1347$ ± $0.0138$  
                               0.50    $0.1347$ ± $0.0138$  
closed-form LR        mad      0.00    $0.0737$ ± $0.0016$  
                               0.05    $0.0747$ ± $0.0013$  
                               0.50    $0.1004$ ± $0.0025$  
            NN        mad      0.00    $0.0358$ ± $0.0005$  
                               0.05    $0.0389$ ± $0.0004$  
                               0.50    $0.0425$ ± $0.0001$  
finite-diff LR        mad      0.00    $0.1470$ ± $0.0022$  
                               0.05    $0.1455$ ± $0.0026$  
                               0.50    $0.1600$ ± $0.0036$  
            NN        mad      0.00    $0.0544$ ± $0.0041$  
                               0.05    $0.0694$ ± $0.0081$  
                               0.50    $0.0618$ ± $0.0088$


------ 2. Prediction MSE (mean ± std) ------


Alpha                                                     0.5  \
Algorithm   Predictor Fairness Lambda                           
FPTO        LR        mad      0.00     $208.8934$ ± $0.8357$   
                               0.05     $208.8758$ ± $0.8303$   
                               0.50     $209.2880$ ± $1.1204$   
            NN        mad      0.00      $35.4627$ ± $1.2644$   
                               0.05      $35.3691$ ± $1.2873$   
                               0.50      $35.2676$ ± $2.0286$   
FoldOpt     LR        mad      0.00    $364.0685$ ± $15.3294$   
                               0.05    $364.0685$ ± $15.3294$   
                               0.50    $364.0685$ ± $15.3294$   
            NN        mad      0.00    $363.7616$ ± $15.4504$   
                               0.05    $363.7616$ ± $15.4504$   
                               0.50    $363.7616$ ± $15.4504$   
closed-form LR        mad      0.00     $262.6600$ ± $2.2327$   
                               0.05     $253.9755$ ± $0.9030$   
                               0.50     $253.8507$ ± $0.8424$   
            NN        mad      0.00    $207.9338$ ± $12.7443$   
                               0.05     $218.2368$ ± $9.4937$   
                               0.50     $220.4741$ ± $9.3006$   
finite-diff LR        mad      0.00     $319.2976$ ± $0.8805$   
                               0.05     $300.2613$ ± $1.2245$   
                               0.50     $300.2064$ ± $1.2518$   
            NN        mad      0.00    $246.7065$ ± $41.6353$   
                               0.05     $257.2754$ ± $7.6626$   
                               0.50     $257.7810$ ± $7.9979$   

Alpha                                                     2.0  
Algorithm   Predictor Fairness Lambda                          
FPTO        LR        mad      0.00     $208.8934$ ± $0.8357$  
                               0.05     $208.8758$ ± $0.8303$  
                               0.50     $209.2880$ ± $1.1204$  
            NN        mad      0.00      $35.4627$ ± $1.2644$  
                               0.05      $35.3691$ ± $1.2873$  
                               0.50      $35.2676$ ± $2.0286$  
FoldOpt     LR        mad      0.00    $362.8136$ ± $15.3300$  
                               0.05    $352.4806$ ± $19.2327$  
                               0.50    $352.4806$ ± $19.2327$  
            NN        mad      0.00    $366.9125$ ± $18.7837$  
                               0.05    $328.9272$ ± $22.5315$  
                               0.50    $328.9056$ ± $22.5657$  
closed-form LR        mad      0.00     $239.5040$ ± $1.5876$  
                               0.05     $236.3998$ ± $1.1029$  
                               0.50     $222.0302$ ± $0.1822$  
            NN        mad      0.00     $119.3416$ ± $2.8384$  
                               0.05     $112.0740$ ± $3.9208$  
                               0.50     $106.9212$ ± $3.0555$  
finite-diff LR        mad      0.00     $296.5613$ ± $2.6815$  
                               0.05     $301.8743$ ± $1.6218$  
                               0.50     $282.6524$ ± $3.9307$  
            NN        mad      0.00    $188.4053$ ± $32.0814$  
                               0.05     $225.3377$ ± $1.1809$  
                               0.50    $190.4567$ ± $13.8893$


------ 3. Prediction Fairness (mean ± std) ------


Alpha                                                    0.5  \
Algorithm   Predictor Fairness Lambda                          
FPTO        LR        mad      0.00     $28.6400$ ± $0.3371$   
                               0.05     $28.6206$ ± $0.3259$   
                               0.50     $28.7634$ ± $0.4367$   
            NN        mad      0.00      $1.7344$ ± $0.1045$   
                               0.05      $1.8268$ ± $0.0954$   
                               0.50      $1.8322$ ± $0.3261$   
FoldOpt     LR        mad      0.00     $91.3055$ ± $8.1582$   
                               0.05     $91.3055$ ± $8.1582$   
                               0.50     $91.3055$ ± $8.1582$   
            NN        mad      0.00     $91.8474$ ± $8.1903$   
                               0.05     $91.8474$ ± $8.1903$   
                               0.50     $91.8474$ ± $8.1903$   
closed-form LR        mad      0.00     $48.2703$ ± $0.6530$   
                               0.05     $37.6696$ ± $0.5499$   
                               0.50     $37.7932$ ± $0.6163$   
            NN        mad      0.00     $24.5686$ ± $3.0886$   
                               0.05     $30.5865$ ± $4.0275$   
                               0.50     $31.9948$ ± $4.4498$   
finite-diff LR        mad      0.00     $63.6933$ ± $0.4787$   
                               0.05     $53.0269$ ± $0.1726$   
                               0.50     $53.0252$ ± $0.2076$   
            NN        mad      0.00    $39.4929$ ± $11.4958$   
                               0.05     $26.3016$ ± $1.3442$   
                               0.50     $26.5533$ ± $1.2669$   

Alpha                                                    2.0  
Algorithm   Predictor Fairness Lambda                         
FPTO        LR        mad      0.00     $28.6400$ ± $0.3371$  
                               0.05     $28.6206$ ± $0.3259$  
                               0.50     $28.7634$ ± $0.4367$  
            NN        mad      0.00      $1.7344$ ± $0.1045$  
                               0.05      $1.8268$ ± $0.0954$  
                               0.50      $1.8322$ ± $0.3261$  
FoldOpt     LR        mad      0.00     $88.9219$ ± $9.9793$  
                               0.05    $77.1218$ ± $11.1481$  
                               0.50    $77.1218$ ± $11.1481$  
            NN        mad      0.00     $92.0644$ ± $6.9585$  
                               0.05    $58.8679$ ± $12.0956$  
                               0.50    $58.8678$ ± $12.0956$  
closed-form LR        mad      0.00     $58.5934$ ± $0.5557$  
                               0.05     $53.0743$ ± $0.2425$  
                               0.50     $35.6152$ ± $0.4845$  
            NN        mad      0.00     $32.4739$ ± $0.4682$  
                               0.05     $37.4980$ ± $2.8442$  
                               0.50     $21.1977$ ± $2.9267$  
finite-diff LR        mad      0.00     $60.5401$ ± $0.0245$  
                               0.05     $63.0944$ ± $1.6179$  
                               0.50     $50.8770$ ± $2.1584$  
            NN        mad      0.00     $51.0395$ ± $6.1237$  
                               0.05     $42.6929$ ± $3.8612$  
                               0.50     $30.2942$ ± $0.3262$


------ 4. Training Time (mean ± std) ------


Alpha                                                   0.5  \
Algorithm   Predictor Fairness Lambda                         
FPTO        LR        mad      0.00     $1.8909$ ± $0.4243$   
                               0.05     $2.3208$ ± $0.0785$   
                               0.50     $2.3467$ ± $0.0247$   
            NN        mad      0.00     $1.2987$ ± $0.0048$   
                               0.05     $1.2830$ ± $0.0500$   
                               0.50     $1.3070$ ± $0.0671$   
FoldOpt     LR        mad      0.00     $5.4392$ ± $0.0613$   
                               0.05     $5.3969$ ± $0.0515$   
                               0.50     $5.3300$ ± $0.0408$   
            NN        mad      0.00     $5.5577$ ± $0.0307$   
                               0.05     $5.5588$ ± $0.0120$   
                               0.50     $5.4976$ ± $0.0547$   
closed-form LR        mad      0.00     $8.9967$ ± $0.0408$   
                               0.05     $3.1345$ ± $0.0094$   
                               0.50     $3.0887$ ± $0.2231$   
            NN        mad      0.00     $7.6495$ ± $0.0986$   
                               0.05     $3.1456$ ± $0.1892$   
                               0.50     $3.2721$ ± $0.0649$   
finite-diff LR        mad      0.00    $85.5750$ ± $1.0894$   
                               0.05    $34.8349$ ± $0.0017$   
                               0.50    $34.1850$ ± $0.3900$   
            NN        mad      0.00    $78.2144$ ± $4.0072$   
                               0.05    $30.2685$ ± $0.3014$   
                               0.50    $30.1630$ ± $0.2920$   

Alpha                                                    2.0  
Algorithm   Predictor Fairness Lambda                         
FPTO        LR        mad      0.00      $2.2822$ ± $0.0001$  
                               0.05      $2.3862$ ± $0.1126$  
                               0.50      $2.5401$ ± $0.2606$  
            NN        mad      0.00      $1.2996$ ± $0.0329$  
                               0.05      $1.3325$ ± $0.0338$  
                               0.50      $1.2913$ ± $0.0412$  
FoldOpt     LR        mad      0.00      $5.4296$ ± $0.0474$  
                               0.05      $5.4522$ ± $0.0274$  
                               0.50      $5.5180$ ± $0.0438$  
            NN        mad      0.00      $5.5543$ ± $0.0590$  
                               0.05      $5.4581$ ± $0.0388$  
                               0.50      $5.5451$ ± $0.0407$  
closed-form LR        mad      0.00      $9.0340$ ± $0.1632$  
                               0.05      $8.9340$ ± $0.1853$  
                               0.50      $9.0255$ ± $0.0321$  
            NN        mad      0.00     $10.1872$ ± $0.0213$  
                               0.05     $10.4324$ ± $0.1570$  
                               0.50     $10.2478$ ± $0.1541$  
finite-diff LR        mad      0.00    $100.3663$ ± $1.4406$  
                               0.05     $99.8769$ ± $0.8692$  
                               0.50     $99.5620$ ± $0.7987$  
            NN        mad      0.00    $103.6114$ ± $0.4983$  
                               0.05    $91.8562$ ± $10.4313$  
                               0.50    $102.2925$ ± $0.7698$



💡 To generate other configurations, change the parameters above:
   - group_setting: True (group alpha fairness) / False (individual alpha fairness)
   - fairness_type: 'mad' or 'atkinson'
   - alpha_values: [0.5, 2] or custom list


### 📄 Generate LaTeX Tables (for publications)

In [75]:
# Generate ALL LaTeX tables at once
print("🎯 Generating LaTeX code for ALL table configurations...")
print("="*80)

# Generate LaTeX for all configurations
all_latex = table_gen.generate_all_latex_tables(alpha_values=alpha_values)

# Save to file
output_dir = os.path.join(os.path.dirname(RESULTS_DIR), 'latex_tables')
os.makedirs(output_dir, exist_ok=True)

latex_file_path = os.path.join(output_dir, 'results_tables.tex')
with open(latex_file_path, 'w', encoding='utf-8') as f:
    # Add LaTeX preamble for standalone compilation
    preamble = """% LaTeX table code for fairness algorithm results
% Required packages:
% \\usepackage{booktabs}
% \\usepackage{array}

"""
    f.write(preamble + all_latex)

begin_table_count = all_latex.count('\\begin{table}')
print(f"\n💾 LaTeX code saved to: {latex_file_path}")
print(f"📄 Total tables generated: {begin_table_count}")
print("\n✅ All LaTeX tables generated successfully!")
print("\n📋 Use the generated .tex file in your LaTeX document with:")
print("   \\input{results_tables.tex}")

🎯 Generating LaTeX code for ALL table configurations...
🎯 Generating LaTeX code for all publication-ready tables...

📋 Processing: MAD fairness, group alpha fairness objective

📋 Processing: MAD fairness, individual alpha fairness objective

📋 Processing: MAD fairness, individual alpha fairness objective

📋 Processing: ATKINSON fairness, group alpha fairness objective

📋 Processing: ATKINSON fairness, group alpha fairness objective

📋 Processing: ATKINSON fairness, individual alpha fairness objective

✅ Generated LaTeX for 16 tables

💾 LaTeX code saved to: E:\myREPO\Fairness-Decision-Focused-Loss\Organized-FDFL\notebooks\latex_tables\results_tables.tex
📄 Total tables generated: 16

✅ All LaTeX tables generated successfully!

📋 Use the generated .tex file in your LaTeX document with:
   \input{results_tables.tex}

📋 Processing: ATKINSON fairness, individual alpha fairness objective

✅ Generated LaTeX for 16 tables

💾 LaTeX code saved to: E:\myREPO\Fairness-Decision-Focused-Loss\Organize

In [76]:
# 🎯 Generate Focused Tables for Main Results Section

def generate_focused_comparison_tables():
    """Generate aggregated comparison tables by alpha value."""
    
    print("🎯 Generating aggregated comparison tables by alpha value...")
    print("="*80)
    
    # Settings for focused comparison
    fairness_type = 'mad'
    predictor = 'NN'
    group_setting = True  # Group alpha fairness objective
    lambda_values = [0, 0.5]  # No fairness vs fairness penalty
    alpha_values = [0.5, 2.0]  # Risk aversion parameters
    
    print(f"📋 Focused Settings:")
    print(f"   • Fairness: {fairness_type.upper()}")
    print(f"   • Predictor: {predictor}")
    print(f"   • Objective: Group alpha fairness")
    print(f"   • Lambda values: {lambda_values}")
    print(f"   • Alpha values: {alpha_values}")
    print("="*80)
    
    # Filter data for focused comparison
    df = master_df[
        (master_df['Group'] == group_setting) &
        (master_df['Fairness'].str.contains(fairness_type, case=False, na=False)) &
        (master_df['Predictor'] == predictor) &
        (master_df['Lambda'].isin(lambda_values)) &
        (master_df['Alpha'].isin(alpha_values))
    ].copy()
    
    if df.empty:
        print("⚠️  No data found for specified filters")
        return
    
    # Create Algorithm column for cleaner names
    df['Algorithm'] = df['Method']
    two_stage_mask = df['Method'] == 'Two-Stage'
    if two_stage_mask.any():
        df.loc[two_stage_mask, 'Algorithm'] = df.loc[two_stage_mask, 'Grad Method']
    
    # Special handling for FoldOpt MSE calculation
    n0, n1 = 4422, 578
    foldopt_mask = (df['Method'] == 'FoldOpt') & (df['Prediction MSE mean'].isna())
    if not df.loc[foldopt_mask].empty:
        g0_mse_mean = df.loc[foldopt_mask, 'G0 MSE mean']
        g1_mse_mean = df.loc[foldopt_mask, 'G1 MSE mean']
        df.loc[foldopt_mask, 'Prediction MSE mean'] = (g0_mse_mean * n0 + g1_mse_mean * n1) / (n0 + n1)
        
        g0_mse_std = df.loc[foldopt_mask, 'G0 MSE std']
        g1_mse_std = df.loc[foldopt_mask, 'G1 MSE std']
        df.loc[foldopt_mask, 'Prediction MSE std'] = np.sqrt((g0_mse_std**2 * n0 + g1_mse_std**2 * n1) / (n0 + n1))
    
    # Store LaTeX code for all aggregated tables
    aggregated_latex_tables = []
    
    # Generate one table for each alpha value
    for alpha in alpha_values:
        print(f"\n📊 Aggregated Results for α = {alpha}")
        print("-" * 60)
        
        # Filter data for this specific alpha
        alpha_df = df[df['Alpha'] == alpha].copy()
        
        if alpha_df.empty:
            print(f"⚠️  No data found for alpha = {alpha}")
            continue
        
        # Create aggregated table
        aggregated_table = create_aggregated_table(alpha_df, alpha)
        
        # Display HTML version
        display(aggregated_table)
        
        # Generate LaTeX for this aggregated table
        latex_code = generate_aggregated_latex_table(aggregated_table, alpha, fairness_type, predictor)
        aggregated_latex_tables.append(latex_code)
        print(f"\n✅ LaTeX code generated for α = {alpha}")
    
    # Save aggregated LaTeX tables to separate file
    output_dir = os.path.join(os.path.dirname(RESULTS_DIR), 'latex_tables')
    os.makedirs(output_dir, exist_ok=True)
    
    aggregated_latex_file = os.path.join(output_dir, 'aggregated_results_tables.tex')
    complete_aggregated_latex = "\n\n".join(aggregated_latex_tables)
    
    with open(aggregated_latex_file, 'w', encoding='utf-8') as f:
        # Add LaTeX preamble
        preamble = """% Aggregated comparison tables by alpha value
% Settings: MAD fairness, NN predictor, Group alpha fairness objective
% Required packages:
% \\usepackage{booktabs}
% \\usepackage{array}

"""
        f.write(preamble + complete_aggregated_latex)
    
    print(f"\n💾 Aggregated LaTeX tables saved to: {aggregated_latex_file}")
    print(f"📄 Total aggregated tables: {len(aggregated_latex_tables)}")
    print("\n🎯 These aggregated tables show all metrics for each alpha value!")

def create_aggregated_table(df: pd.DataFrame, alpha: float) -> pd.DataFrame:
    """Create aggregated table with all metrics for a specific alpha value."""
    
    # Define metrics to include
    metrics = {
        'Decision Regret': 'Decision Regret mean',
        'Prediction MSE': 'Prediction MSE mean',
        'Prediction Fairness': 'Prediction Fairness mean',
        'Training Time': 'Training Time mean'
    }
    
    # Create index: Algorithm and Lambda
    table_index = ['Algorithm', 'Lambda']
    
    # Create aggregated table structure
    aggregated_data = []
    
    # Get unique combinations of Algorithm and Lambda for this alpha
    alpha_df = df[df['Alpha'] == alpha]
    
    for _, row in alpha_df.iterrows():
        algorithm = row['Algorithm']
        lambda_val = row['Lambda']
        
        # Format algorithm name
        if algorithm == 'closed-form':
            algorithm = 'FDFL-CF'
        elif algorithm == 'finite-diff':
            algorithm = 'FDFL-FD'
        elif algorithm == 'FoldOpt':
            algorithm = 'FoldOpt'
        elif algorithm == 'FPTO':
            algorithm = 'FPTO'
        
        # Format lambda value
        if lambda_val == 0:
            lambda_str = "$0$ (no fairness)"
        elif lambda_val == 0.5:
            lambda_str = "$0.5$"
        else:
            lambda_str = f"${lambda_val}$"
        
        # Create row data
        row_data = {
            'Algorithm': algorithm,
            'Lambda': lambda_str
        }
        
        # Add all metrics with mean ± std format
        for metric_name, metric_col in metrics.items():
            mean_val = row[metric_col] if pd.notna(row[metric_col]) else None
            std_col = metric_col.replace('mean', 'std')
            std_val = row[std_col] if std_col in row and pd.notna(row[std_col]) else None
            
            if mean_val is not None:
                if std_val is not None:
                    row_data[metric_name] = f"${mean_val:.4f}$ ± ${std_val:.4f}$"
                else:
                    row_data[metric_name] = f"${mean_val:.4f}$"
            else:
                row_data[metric_name] = "$-$"
        
        aggregated_data.append(row_data)
    
    # Convert to DataFrame
    aggregated_df = pd.DataFrame(aggregated_data)
    
    # Set multi-index for proper formatting
    if not aggregated_df.empty:
        aggregated_df = aggregated_df.set_index(['Algorithm', 'Lambda'])
    
    return aggregated_df

def generate_aggregated_latex_table(table: pd.DataFrame, alpha: float, fairness_type: str, predictor: str) -> str:
    """Generate LaTeX for aggregated comparison table."""
    
    # Create caption and label for aggregated table
    caption = f"Algorithm comparison for $\\alpha = {alpha}$ (MAD fairness, {predictor} predictor, group alpha fairness objective)"
    label = f"tab:aggregated_alpha_{str(alpha).replace('.', '_')}_mad_{predictor.lower()}_group"
    
    # Start LaTeX table
    latex_lines = [
        "\\begin{table}[htbp]",
        "\\centering",
        f"\\caption{{{caption}}}",
        f"\\label{{{label}}}",
        "\\footnotesize",
    ]
    
    # Column specification: Algorithm(l) + Lambda(l) + Metrics(c)
    n_metric_cols = len(table.columns)
    col_spec = "ll" + "c" * n_metric_cols  # Algorithm, Lambda, then centered metric columns
    latex_lines.append(f"\\begin{{tabular}}{{{col_spec}}}")
    latex_lines.append("\\toprule")
    
    # Header row
    header_parts = ["Algorithm", "Lambda"] + list(table.columns)
    header_line = " & ".join(header_parts) + " \\\\"
    latex_lines.append(header_line)
    latex_lines.append("\\midrule")
    
    # Data rows
    for idx, row in table.iterrows():
        # Handle index (Algorithm, Lambda)
        if isinstance(idx, tuple):
            algorithm, lambda_val = idx
        else:
            algorithm, lambda_val = str(idx), ""
        
        # Data values (already formatted with dollar signs)
        data_parts = [str(val) if pd.notna(val) else "$-$" for val in row.values]
        
        # Combine all parts
        all_parts = [algorithm, lambda_val] + data_parts
        row_line = " & ".join(all_parts) + " \\\\"
        latex_lines.append(row_line)
    
    # End table
    latex_lines.extend([
        "\\bottomrule",
        "\\end{tabular}",
        "\\end{table}",
        ""
    ])
    
    return "\n".join(latex_lines)

def generate_focused_latex_table(table: pd.DataFrame, table_name: str, fairness_type: str, predictor: str) -> str:
    """Generate LaTeX for focused comparison table."""
    
    # Create caption and label for focused table
    caption = f"{table_name} comparison (MAD fairness, {predictor} predictor, group alpha fairness objective)"
    label = f"tab:focused_{table_name.lower().replace(' ', '_')}_mad_{predictor.lower()}_group"
    
    # Start LaTeX table
    latex_lines = [
        "\\begin{table}[htbp]",
        "\\centering",
        f"\\caption{{{caption}}}",
        f"\\label{{{label}}}",
        "\\footnotesize",
    ]
    
    # Column specification: Algorithm(l) + Lambda(l) + Alpha values(c)
    n_data_cols = len(table.columns)
    col_spec = "ll" + "c" * n_data_cols  # Algorithm, Lambda, then centered data columns
    latex_lines.append(f"\\begin{{tabular}}{{{col_spec}}}")
    latex_lines.append("\\toprule")
    
    # Header row
    header_parts = ["Algorithm", "Lambda"] + [f"$\\alpha = {col}$" for col in table.columns]
    header_line = " & ".join(header_parts) + " \\\\"
    latex_lines.append(header_line)
    latex_lines.append("\\midrule")
    
    # Data rows
    for idx, row in table.iterrows():
        # Handle index (Algorithm, Lambda)
        if isinstance(idx, tuple):
            algorithm, lambda_val = idx
        else:
            algorithm, lambda_val = str(idx), ""
        
        # Format algorithm names
        if algorithm == 'closed-form':
            algorithm = 'FDFL-CF'
        elif algorithm == 'finite-diff':
            algorithm = 'FDFL-FD'
        elif algorithm == 'FoldOpt':
            algorithm = 'FoldOpt'
        elif algorithm == 'FPTO':
            algorithm = 'FPTO'
        
        # Format lambda value
        if lambda_val == 0:
            lambda_str = "$0$ (no fairness)"
        elif lambda_val == 0.5:
            lambda_str = "$0.5$"
        else:
            lambda_str = f"${lambda_val}$"
        
        # Data values (already formatted with dollar signs)
        data_parts = [str(val) if pd.notna(val) else "$-$" for val in row.values]
        
        # Combine all parts
        all_parts = [algorithm, lambda_str] + data_parts
        row_line = " & ".join(all_parts) + " \\\\"
        latex_lines.append(row_line)
    
    # End table
    latex_lines.extend([
        "\\bottomrule",
        "\\end{tabular}",
        "\\end{table}",
        ""
    ])
    
    return "\n".join(latex_lines)

# Generate the focused comparison tables
generate_focused_comparison_tables()

🎯 Generating aggregated comparison tables by alpha value...
📋 Focused Settings:
   • Fairness: MAD
   • Predictor: NN
   • Objective: Group alpha fairness
   • Lambda values: [0, 0.5]
   • Alpha values: [0.5, 2.0]

📊 Aggregated Results for α = 0.5
------------------------------------------------------------


Decision Regret          Prediction MSE  \
Algorithm Lambda                                                           
FoldOpt   $0$ (no fairness)  $0.1325$ ± $0.0039$  $363.7616$ ± $15.4504$   
          $0.5$              $0.1325$ ± $0.0039$  $363.7616$ ± $15.4504$   
FDFL-CF   $0$ (no fairness)  $0.0189$ ± $0.0012$  $207.9338$ ± $12.7443$   
          $0.5$              $0.0763$ ± $0.0013$   $220.4741$ ± $9.3006$   
FDFL-FD   $0$ (no fairness)  $0.0223$ ± $0.0035$  $246.7065$ ± $41.6353$   
          $0.5$              $0.0810$ ± $0.0007$   $257.7810$ ± $7.9979$   
FPTO      $0$ (no fairness)  $0.0176$ ± $0.0004$    $35.4627$ ± $1.2644$   
          $0.5$              $0.0179$ ± $0.0011$    $35.2676$ ± $2.0286$   

                               Prediction Fairness         Training Time  
Algorithm Lambda                                                          
FoldOpt   $0$ (no fairness)   $91.8474$ ± $8.1903$   $5.5577$ ± $0.0307$  
          $0.5$               $91.8474$ ± $8.1903$   $5.4976$ ± $0.0547$  
FDFL-CF   $0$ (no fairness)   $24.5686$ ± $3.0886$   $7.6495$ ± $0.0986$  
          $0.5$               $31.9948$ ± $4.4498$   $3.2721$ ± $0.0649$  
FDFL-FD   $0$ (no fairness)  $39.4929$ ± $11.4958$  $78.2144$ ± $4.0072$  
          $0.5$               $26.5533$ ± $1.2669$  $30.1630$ ± $0.2920$  
FPTO      $0$ (no fairness)    $1.7344$ ± $0.1045$   $1.2987$ ± $0.0048$  
          $0.5$                $1.8322$ ± $0.3261$   $1.3070$ ± $0.0671$


✅ LaTeX code generated for α = 0.5

📊 Aggregated Results for α = 2.0
------------------------------------------------------------


Decision Regret          Prediction MSE  \
Algorithm Lambda                                                           
FoldOpt   $0$ (no fairness)  $0.1195$ ± $0.0068$  $366.9125$ ± $18.7837$   
          $0.5$              $0.1347$ ± $0.0138$  $328.9056$ ± $22.5657$   
FDFL-CF   $0$ (no fairness)  $0.0358$ ± $0.0005$   $119.3416$ ± $2.8384$   
          $0.5$              $0.0425$ ± $0.0001$   $106.9212$ ± $3.0555$   
FDFL-FD   $0$ (no fairness)  $0.0544$ ± $0.0041$  $188.4053$ ± $32.0814$   
          $0.5$              $0.0618$ ± $0.0088$  $190.4567$ ± $13.8893$   
FPTO      $0$ (no fairness)  $0.0983$ ± $0.0018$    $35.4627$ ± $1.2644$   
          $0.5$              $0.0968$ ± $0.0039$    $35.2676$ ± $2.0286$   

                               Prediction Fairness          Training Time  
Algorithm Lambda                                                           
FoldOpt   $0$ (no fairness)   $92.0644$ ± $6.9585$    $5.5543$ ± $0.0590$  
          $0.5$              $58.8678$ ± $12.0956$    $5.5451$ ± $0.0407$  
FDFL-CF   $0$ (no fairness)   $32.4739$ ± $0.4682$   $10.1872$ ± $0.0213$  
          $0.5$               $21.1977$ ± $2.9267$   $10.2478$ ± $0.1541$  
FDFL-FD   $0$ (no fairness)   $51.0395$ ± $6.1237$  $103.6114$ ± $0.4983$  
          $0.5$               $30.2942$ ± $0.3262$  $102.2925$ ± $0.7698$  
FPTO      $0$ (no fairness)    $1.7344$ ± $0.1045$    $1.2996$ ± $0.0329$  
          $0.5$                $1.8322$ ± $0.3261$    $1.2913$ ± $0.0412$


✅ LaTeX code generated for α = 2.0

💾 Aggregated LaTeX tables saved to: E:\myREPO\Fairness-Decision-Focused-Loss\Organized-FDFL\notebooks\latex_tables\aggregated_results_tables.tex
📄 Total aggregated tables: 2

🎯 These aggregated tables show all metrics for each alpha value!


In [79]:
def prepare_plotting_data():
    """准备用于R绘图的数据格式"""
    
    print("🎯 准备绘图数据...")
    print("="*80)
    
    # 设置筛选条件
    fairness_type = 'mad'
    predictor = 'NN'
    group_setting = True
    lambda_values = [0, 0.5]  # 无公平性约束 vs 有公平性约束
    alpha_values = [0.5, 2.0]
    
    print(f"📋 绘图设置:")
    print(f"   • 公平性类型: {fairness_type.upper()}")
    print(f"   • 预测器: {predictor}")
    print(f"   • 目标函数: 群体alpha公平性")
    print(f"   • Lambda值: {lambda_values}")
    print(f"   • Alpha值: {alpha_values}")
    
    # 筛选数据
    df = master_df[
        (master_df['Group'] == group_setting) &
        (master_df['Fairness'].str.contains(fairness_type, case=False, na=False)) &
        (master_df['Predictor'] == predictor) &
        (master_df['Lambda'].isin(lambda_values)) &
        (master_df['Alpha'].isin(alpha_values))
    ].copy()
    
    if df.empty:
        print("⚠️  没有找到符合条件的数据")
        return None
    
    # 创建算法列
    df['Algorithm'] = df['Method']
    two_stage_mask = df['Method'] == 'Two-Stage'
    if two_stage_mask.any():
        df.loc[two_stage_mask, 'Algorithm'] = df.loc[two_stage_mask, 'Grad Method']
    
    # 处理FoldOpt的MSE计算
    n0, n1 = 4422, 578
    foldopt_mask = (df['Method'] == 'FoldOpt') & (df['Prediction MSE mean'].isna())
    if not df.loc[foldopt_mask].empty:
        g0_mse_mean = df.loc[foldopt_mask, 'G0 MSE mean']
        g1_mse_mean = df.loc[foldopt_mask, 'G1 MSE mean']
        df.loc[foldopt_mask, 'Prediction MSE mean'] = (g0_mse_mean * n0 + g1_mse_mean * n1) / (n0 + n1)
        
        g0_mse_std = df.loc[foldopt_mask, 'G0 MSE std']
        g1_mse_std = df.loc[foldopt_mask, 'G1 MSE std']
        df.loc[foldopt_mask, 'Prediction MSE std'] = np.sqrt((g0_mse_std**2 * n0 + g1_mse_std**2 * n1) / (n0 + n1))
    
    # 清理算法名称
    algorithm_mapping = {
        'closed-form': 'FDFL-CF',
        'finite-diff': 'FDFL-FD',
        'FoldOpt': 'FoldOpt',
        'FPTO': 'FPTO'
    }
    df['Algorithm'] = df['Algorithm'].map(algorithm_mapping).fillna(df['Algorithm'])
    
    # 创建Lambda标签
    df['Lambda_Label'] = df['Lambda'].map({
        0: 'No Fairness (λ=0)',
        0.5: 'With Fairness (λ=0.5)'
    })
    
    # 定义要绘制的指标
    metrics = {
        'Decision_Regret': ('Decision Regret mean', 'Decision Regret std'),
        'Prediction_MSE': ('Prediction MSE mean', 'Prediction MSE std'),
        'Prediction_Fairness': ('Prediction Fairness mean', 'Prediction Fairness std'),
        'Training_Time': ('Training Time mean', 'Training Time std')
    }
    
    # 为每个指标准备数据
    plot_data = {}
    
    for metric_name, (mean_col, std_col) in metrics.items():
        print(f"\n📊 准备 {metric_name} 数据...")
        
        # 选择需要的列
        plot_df = df[['Algorithm', 'Alpha', 'Lambda', 'Lambda_Label', mean_col, std_col]].copy()
        plot_df = plot_df.dropna(subset=[mean_col])
        
        # 重命名列以便R使用
        plot_df.rename(columns={
            mean_col: 'Value',
            std_col: 'StdDev'
        }, inplace=True)
        
        # 添加指标名称
        plot_df['Metric'] = metric_name
        
        plot_data[metric_name] = plot_df
        print(f"   ✓ 数据形状: {plot_df.shape}")
        print(f"   ✓ 算法: {plot_df['Algorithm'].unique()}")
        print(f"   ✓ Alpha值: {plot_df['Alpha'].unique()}")
    
    return plot_data

def export_plotting_data_to_csv(plot_data):
    """将绘图数据导出为CSV文件供R使用"""
    
    if plot_data is None:
        return
    
    # 创建输出目录
    output_dir = os.path.join(os.path.dirname(RESULTS_DIR), 'plotting_data')
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"\n💾 导出绘图数据到: {output_dir}")
    
    # 导出每个指标的数据
    for metric_name, df in plot_data.items():
        csv_file = os.path.join(output_dir, f'{metric_name.lower()}_data.csv')
        df.to_csv(csv_file, index=False)
        print(f"   ✓ {csv_file}")
    
    # 合并所有数据到一个文件
    combined_df = pd.concat(plot_data.values(), ignore_index=True)
    combined_csv = os.path.join(output_dir, 'combined_metrics_data.csv')
    combined_df.to_csv(combined_csv, index=False)
    print(f"   ✓ {combined_csv} (合并文件)")
    
    return output_dir

def generate_r_plotting_code(output_dir):
    """生成R绘图代码"""
    
    r_code = '''
# R代码：绘制算法比较图表
# 需要的包
library(ggplot2)
library(dplyr)
library(readr)
library(scales)

# 设置工作目录（根据需要修改路径）
# setwd("your_path_here")

# 读取数据
data <- read_csv("combined_metrics_data.csv")

# 创建绘图函数
create_comparison_plot <- function(data, metric_name, y_label, title_suffix = "") {
  
  # 筛选特定指标的数据
  plot_data <- data %>% 
    filter(Metric == metric_name) %>%
    mutate(
      Algorithm = factor(Algorithm, levels = c("FDFL-CF", "FDFL-FD", "FoldOpt", "FPTO")),
      Alpha = factor(Alpha, levels = c(0.5, 2.0)),
      Lambda_Label = factor(Lambda_Label, levels = c("No Fairness (λ=0)", "With Fairness (λ=0.5)"))
    )
  
  # 创建图表
  p <- ggplot(plot_data, aes(x = Algorithm, y = Value, fill = Lambda_Label)) +
    geom_col(position = position_dodge(width = 0.8), width = 0.7, alpha = 0.8) +
    geom_errorbar(
      aes(ymin = Value - StdDev, ymax = Value + StdDev),
      position = position_dodge(width = 0.8),
      width = 0.3,
      linewidth = 0.5
    ) +
    facet_wrap(~ paste("α =", Alpha), scales = "free_y", ncol = 2) +
    scale_fill_manual(
      values = c("No Fairness (λ=0)" = "#3498db", "With Fairness (λ=0.5)" = "#e74c3c"),
      name = "Fairness Setting"
    ) +
    labs(
      title = paste("Algorithm Comparison:", gsub("_", " ", metric_name), title_suffix),
      x = "Algorithm",
      y = y_label,
      caption = "Error bars represent ± 1 standard deviation"
    ) +
    theme_minimal() +
    theme(
      plot.title = element_text(size = 14, face = "bold", hjust = 0.5),
      axis.text.x = element_text(angle = 45, hjust = 1, size = 10),
      axis.text.y = element_text(size = 10),
      axis.title = element_text(size = 12, face = "bold"),
      legend.position = "bottom",
      legend.title = element_text(size = 11, face = "bold"),
      legend.text = element_text(size = 10),
      strip.text = element_text(size = 11, face = "bold"),
      panel.grid.minor = element_blank(),
      panel.border = element_rect(color = "grey80", fill = NA, linewidth = 0.5)
    )
  
  return(p)
}

# 为每个指标创建图表
metrics_config <- list(
  "Decision_Regret" = list(y_label = "Decision Regret", title_suffix = ""),
  "Prediction_MSE" = list(y_label = "Prediction MSE", title_suffix = ""),
  "Prediction_Fairness" = list(y_label = "Prediction Fairness", title_suffix = ""),
  "Training_Time" = list(y_label = "Training Time (seconds)", title_suffix = "")
)

# 生成所有图表
plots <- list()
for (metric in names(metrics_config)) {
  config <- metrics_config[[metric]]
  plots[[metric]] <- create_comparison_plot(data, metric, config$y_label, config$title_suffix)
  
  # 保存图表
  ggsave(
    filename = paste0(tolower(metric), "_comparison.png"),
    plot = plots[[metric]],
    width = 12, height = 8, dpi = 300, bg = "white"
  )
  
  # 也保存PDF版本
  ggsave(
    filename = paste0(tolower(metric), "_comparison.pdf"),
    plot = plots[[metric]],
    width = 12, height = 8, device = "pdf"
  )
}

# 显示决策遗憾图表
print(plots$Decision_Regret)

# 创建综合比较图表（所有指标在一个图中）
create_combined_plot <- function(data) {
  
  # 标准化数据用于比较
  normalized_data <- data %>%
    group_by(Metric) %>%
    mutate(
      Normalized_Value = (Value - min(Value, na.rm = TRUE)) / (max(Value, na.rm = TRUE) - min(Value, na.rm = TRUE)),
      Algorithm = factor(Algorithm, levels = c("FDFL-CF", "FDFL-FD", "FoldOpt", "FPTO")),
      Alpha = factor(Alpha, levels = c(0.5, 2.0)),
      Lambda_Label = factor(Lambda_Label, levels = c("No Fairness (λ=0)", "With Fairness (λ=0.5)"))
    )
  
  p <- ggplot(normalized_data, aes(x = Algorithm, y = Normalized_Value, fill = Lambda_Label)) +
    geom_col(position = position_dodge(width = 0.8), width = 0.7, alpha = 0.8) +
    facet_grid(Metric ~ paste("α =", Alpha), scales = "free_y") +
    scale_fill_manual(
      values = c("No Fairness (λ=0)" = "#3498db", "With Fairness (λ=0.5)" = "#e74c3c"),
      name = "Fairness Setting"
    ) +
    labs(
      title = "Normalized Algorithm Comparison Across All Metrics",
      x = "Algorithm",
      y = "Normalized Value (0-1 scale)",
      caption = "Values normalized within each metric for comparison"
    ) +
    theme_minimal() +
    theme(
      plot.title = element_text(size = 14, face = "bold", hjust = 0.5),
      axis.text.x = element_text(angle = 45, hjust = 1, size = 9),
      axis.text.y = element_text(size = 9),
      axis.title = element_text(size = 12, face = "bold"),
      legend.position = "bottom",
      legend.title = element_text(size = 11, face = "bold"),
      legend.text = element_text(size = 10),
      strip.text = element_text(size = 10, face = "bold"),
      panel.grid.minor = element_blank()
    )
  
  return(p)
}

# 创建并保存综合图表
combined_plot <- create_combined_plot(data)
ggsave("combined_metrics_comparison.png", combined_plot, width = 16, height = 12, dpi = 300, bg = "white")
ggsave("combined_metrics_comparison.pdf", combined_plot, width = 16, height = 12, device = "pdf")

print("✅ 所有图表已生成并保存！")
'''
    
    # 保存R代码到文件
    r_file = os.path.join(output_dir, 'generate_plots.R')
    with open(r_file, 'w', encoding='utf-8') as f:
        f.write(r_code)
    
    print(f"\n📝 R绘图代码已保存到: {r_file}")
    print("\n🎯 使用说明:")
    print("1. 在R中设置工作目录到plotting_data文件夹")
    print("2. 运行 source('generate_plots.R')")
    print("3. 图表将自动生成并保存为PNG和PDF格式")
    
    return r_file

# 执行数据准备和导出
print("🎯 开始准备R绘图数据...")
plot_data = prepare_plotting_data()

if plot_data:
    output_dir = export_plotting_data_to_csv(plot_data)
    r_file = generate_r_plotting_code(output_dir)
    
    print(f"\n✅ 数据准备完成！")
    print(f"📁 输出目录: {output_dir}")
    print(f"📊 数据文件: combined_metrics_data.csv")
    print(f"📝 R代码文件: generate_plots.R")
    
    # 显示数据样本
    print(f"\n📋 数据样本预览:")
    combined_df = pd.concat(plot_data.values(), ignore_index=True)
    display(combined_df.head(10))
    
    print(f"\n📊 数据统计:")
    print(f"   • 总行数: {len(combined_df)}")
    print(f"   • 指标类型: {combined_df['Metric'].unique()}")
    print(f"   • 算法: {combined_df['Algorithm'].unique()}")
    print(f"   • Alpha值: {combined_df['Alpha'].unique()}")
    print(f"   • Lambda设置: {combined_df['Lambda_Label'].unique()}")


🎯 开始准备R绘图数据...
🎯 准备绘图数据...
📋 绘图设置:
   • 公平性类型: MAD
   • 预测器: NN
   • 目标函数: 群体alpha公平性
   • Lambda值: [0, 0.5]
   • Alpha值: [0.5, 2.0]

📊 准备 Decision_Regret 数据...
   ✓ 数据形状: (16, 7)
   ✓ 算法: ['FoldOpt' 'FDFL-CF' 'FDFL-FD' 'FPTO']
   ✓ Alpha值: [0.5 2. ]

📊 准备 Prediction_MSE 数据...
   ✓ 数据形状: (16, 7)
   ✓ 算法: ['FoldOpt' 'FDFL-CF' 'FDFL-FD' 'FPTO']
   ✓ Alpha值: [0.5 2. ]

📊 准备 Prediction_Fairness 数据...
   ✓ 数据形状: (16, 7)
   ✓ 算法: ['FoldOpt' 'FDFL-CF' 'FDFL-FD' 'FPTO']
   ✓ Alpha值: [0.5 2. ]

📊 准备 Training_Time 数据...
   ✓ 数据形状: (16, 7)
   ✓ 算法: ['FoldOpt' 'FDFL-CF' 'FDFL-FD' 'FPTO']
   ✓ Alpha值: [0.5 2. ]

💾 导出绘图数据到: E:\myREPO\Fairness-Decision-Focused-Loss\Organized-FDFL\notebooks\plotting_data
   ✓ E:\myREPO\Fairness-Decision-Focused-Loss\Organized-FDFL\notebooks\plotting_data\decision_regret_data.csv
   ✓ E:\myREPO\Fairness-Decision-Focused-Loss\Organized-FDFL\notebooks\plotting_data\prediction_mse_data.csv
   ✓ E:\myREPO\Fairness-Decision-Focused-Loss\Organized-FDFL\notebooks\plotting_dat

,Algorithm,Alpha,Lambda,Lambda_Label,Value,StdDev,Metric
0,FoldOpt,0.5,0.0,No Fairness (λ=0),0.132487,0.003942,Decision_Regret
1,FoldOpt,0.5,0.5,With Fairness (λ=0.5),0.132487,0.003942,Decision_Regret
2,FoldOpt,2.0,0.0,No Fairness (λ=0),0.119466,0.006823,Decision_Regret
3,FoldOpt,2.0,0.5,With Fairness (λ=0.5),0.134672,0.013759,Decision_Regret
4,FDFL-CF,0.5,0.0,No Fairness (λ=0),0.018903,0.001192,Decision_Regret
5,FDFL-CF,0.5,0.5,With Fairness (λ=0.5),0.076344,0.001319,Decision_Regret
6,FDFL-CF,2.0,0.0,No Fairness (λ=0),0.035783,0.000505,Decision_Regret
7,FDFL-CF,2.0,0.5,With Fairness (λ=0.5),0.042493,0.000117,Decision_Regret
8,FDFL-FD,0.5,0.0,No Fairness (λ=0),0.022311,0.003528,Decision_Regret
9,FDFL-FD,0.5,0.5,With Fairness (λ=0.5),0.081019,0.000676,Decision_Regret



📊 数据统计:
   • 总行数: 64
   • 指标类型: ['Decision_Regret' 'Prediction_MSE' 'Prediction_Fairness' 'Training_Time']
   • 算法: ['FoldOpt' 'FDFL-CF' 'FDFL-FD' 'FPTO']
   • Alpha值: [0.5 2. ]
   • Lambda设置: ['No Fairness (λ=0)' 'With Fairness (λ=0.5)']
